<a href="https://colab.research.google.com/github/khal-drog0/NLP/blob/main/english_to_hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
! unzip /content/drive/MyDrive/Hindi_English_Truncated_Corpus.csv.zip

Archive:  /content/drive/MyDrive/Hindi_English_Truncated_Corpus.csv.zip
  inflating: Hindi_English_Truncated_Corpus.csv  


In [ ]:
! ls

drive  Hindi_English_Truncated_Corpus.csv  sample_data


In [ ]:
# Ref: https://www.kaggle.com/code/aiswaryaramachandran/english-to-hindi-neural-machine-translation/notebook

In [ ]:
import pandas as pd

lines = pd.read_csv('Hindi_English_Truncated_Corpus.csv')

In [ ]:
lines.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [ ]:
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import numpy as np

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [ ]:
lines.source.value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [ ]:
lines = lines[lines['source'] == 'ted']

In [ ]:
lines.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
23,ted,This changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
30,ted,"And you can see, this LED is going to glow.","और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।"
32,ted,to turn on the lights or to bring him a glass ...,"लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,"
35,ted,Can you imagine saying that?,क्या आप ये कल्पना कर सकते है


In [ ]:
pd.isnull(lines).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [ ]:
lines = lines[~pd.isnull(lines['english_sentence'])]

In [ ]:
lines.drop_duplicates(inplace = True)

In [ ]:
lines = lines.sample(n = 25000, random_state = 42)
lines.shape

(25000, 3)

In [ ]:
# Converting all the characters to lowercase
lines.english_sentence = lines.english_sentence.apply(lambda x : x.lower())
lines.hindi_sentence = lines.hindi_sentence.apply(lambda x : x.lower())

In [ ]:
# Removing quotes
lines.english_sentence = lines.english_sentence.apply(lambda x : re.sub("'", '', x))
lines.hindi_sentence = lines.hindi_sentence.apply(lambda x : re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # set of all special characters

# Removing all the special characters
lines.english_sentence = lines.english_sentence.apply(lambda x : ''.join(ch for ch in x if ch not in exclude))
lines.hindi_sentence = lines.hindi_sentence.apply(lambda x : ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Removing all the numbers from the text
remove_digits = str.maketrans('', '', digits)

lines.englis_sentence = lines.english_sentence.apply(lambda x : x.translate(remove_digits))
lines.hindi_sentence = lines.hindi_sentence.apply(lambda x : x.translate(remove_digits))

lines.hindi_sentence =lines.hindi_sentence.apply(lambda x : re.sub('[२३०८१५७९४६]', '', x))

# Remove extra spaces
lines.english_sentence = lines.english_sentence.apply(lambda x : x.strip())
lines.hindi_sentence = lines.hindi_sentence.apply(lambda x : x.strip())

lines.english_sentence = lines.english_sentence.apply(lambda x : re.sub(' +', ' ', x))
lines.hindi_sentence = lines.hindi_sentence.apply(lambda x : re.sub(' +', ' ', x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


In [ ]:
# Adding start and end tokens to target sentences
lines.hindi_sentence = lines.hindi_sentence.apply(lambda x : 'START_'+ x +' _END')

In [ ]:
lines.head()

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_हम अभी तक नहीं जानते हैं कि उसके मातापित...
85038,ted,no keyboard,START_कोई कुंजीपटल नहीं _END
58018,ted,but as far as being a performer,START_लेकिन एक कलाकार होने के साथ _END
74470,ted,and this particular balloon,START_और यह खास गुब्बारा _END
122330,ted,and its not as hard as you think integrate cli...,START_और जितना आपको लगता है यह उतना कठिन नहीं ...


In [ ]:
# Get English to Hindi Vocabulary
all_eng_words = set()
all_hindi_words = set()

for eng in lines.english_sentence:
  for word in eng.split():
    if word not in all_eng_words:
      all_eng_words.add(word)

for hin in lines.hindi_sentence:
  for word in hin.split():
    if word not in all_hindi_words:
      all_hindi_words.add(word)

In [ ]:
len(all_eng_words), len(all_hindi_words)

(14362, 20054)

In [ ]:
lines['length_eng_sentence'] = lines.english_sentence.apply(lambda x : len(x.split(' ')))
lines['length_hin_sentence'] = lines.hindi_sentence.apply(lambda x : len(x.split(' ')))

In [ ]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,we still dont know who her parents are who she is,START_हम अभी तक नहीं जानते हैं कि उसके मातापित...,11,15
85038,ted,no keyboard,START_कोई कुंजीपटल नहीं _END,2,4
58018,ted,but as far as being a performer,START_लेकिन एक कलाकार होने के साथ _END,7,7
74470,ted,and this particular balloon,START_और यह खास गुब्बारा _END,4,5
122330,ted,and its not as hard as you think integrate cli...,START_और जितना आपको लगता है यह उतना कठिन नहीं ...,16,19


In [ ]:
# Finding how many english sentences have length greater than 30

lines[lines.length_eng_sentence > 30].shape

(0, 5)

In [ ]:
lines[lines.length_hin_sentence > 30].shape

(0, 5)

In [ ]:
lines = lines[lines.length_eng_sentence <= 20]
lines = lines[lines.length_hin_sentence <= 20]

In [ ]:
lines.shape

(24867, 5)

In [ ]:
print('Max length of hindi sentence:', max(lines.length_hin_sentence))
print('Max length of english sentence:', max(lines.length_eng_sentence))

Max length of hindi sentence: 20
Max length of english sentence: 20


In [ ]:
max_length_src = max(lines.length_hin_sentence)
max_length_tar = max(lines.length_eng_sentence)

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(14362, 20054)

In [ ]:
num_decoder_tokens += 1

In [ ]:
input_token_index = dict([(word, i + 1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i + 1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
4295,ted,to doing just one thing very well,START_सिर्फ़ एक काम को ढँग से करने में। _END,7,9
39586,ted,of what we do,START_हमारे काम के बारे में _END,4,6
11344,ted,and theres a guy you know in the back”,START_तो देखा कि एक आदमी है तुम जानती हो मेरे ...,9,12
116653,ted,everything else i could possible want to wear,START_जो कुछ भी मुझे पहनने का मन होगा _END,8,9
49695,ted,and i said “okay ill go to the old man in village,START_और मैने कहा “ठीक है मैं गाँव के बूढे के ...,12,14
53758,ted,labels like “regulars” and “honors” resonate,START_“नियमित” और “सम्मान” की तरह के लेबल गूंज...,6,10
19025,ted,i knew what the image of india was then if the...,START_मुझे मालुम था की भारत की क्या तस्वीर है ...,16,17
87804,ted,every row is a student,START_हर लेटी पंक्ति एक स्टूडेंट है। _END,5,7
124171,ted,but was india ready for it,START_पर क्या भारत इसके लिए तैयार था _END,6,8
66044,ted,and we are much more confident now than we wer...,START_और दो साल पहले की तुलना में हम और अधिक आ...,13,13


In [ ]:
# Splitting the data into train and test
x, y = lines.english_sentence, lines.hindi_sentence
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train.shape, x_test.shape

((19893,), (4974,))

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.4, random_state = 42)
x_train.shape, x_val.shape

((11935,), (7958,))

In [ ]:
x_train.to_pickle('x_train.pkl')
x_test.to_pickle('x_test.pkl')
x_val.to_pickle('x_val.pkl')

In [ ]:
# Generate batch of data 
def generate_batch(x = x_train, y = y_train, batch_size = 128):
  while True:
    for j in range(0, len(x), batch_size):
      encoder_input_data = np.zeros((batch_size, max_length_src), dtype = 'float32')
      decoder_input_data = np.zeros((batch_size, max_length_tar), dtype = 'float32')
      decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens), dtype = 'float32')
      for i, (input_text, target_text) in enumerate(zip(x[j : j + batch_size], y[j : j + batch_size])):
        for t, word in enumerate(input_text.split()):
          encoder_input_data[i, t] = input_token_index[word] # Encoder input sequence
          for t, word in enumerate(target_text.split()):
            if t < len(target_text.split()) - 1:
              decoder_input_data[i, t] = target_token_index[word] # Decoder input sequence
            if t > 0:
              # Decoder target sequence (one hot encoded)
              # does not include START_ token
              # Offset by one timestep
              decoder_target_data[i, t - 1, target_token_index[word]] = 1
              yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
# Encoder - Decoder Architecture
latent_dim = 300

In [ ]:
from tensorflow import keras

In [ ]:
# Encoder 
encoder_inputs = Input(shape = (None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state = True, dropout = 0.2)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard 'encoder_output' and only keep the states so that we can encode and decode successfully.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using 'encoder_states' as initial state.
decoder_inputs = Input(shape = (None, ))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the traininng model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True,
                    dropout = 0.2)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
import tensorflow as tf

rlronp = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    mode = 'min',
    min_lr = 1e-8,
    patience = 2,
    verbose = 1
    )

m = [
    'acc', 
    tf.keras.metrics.Precision(), 
    tf.keras.metrics.Recall(), 
    tf.keras.metrics.AUC()
    ]

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 1e-5)
model.compile(optimizer = opt, loss = 'cosine_similarity', metrics = m)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    4308600     ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 300)    6016500     ['input_4[0][0]']                
                                                                                            

In [ ]:
train_samples = len(x_train)
val_samples = len(x_val)
batch_size = 128
epochs = 25

In [ ]:
with tf.device('/gpu:0'):
  model.fit_generator(generator = generate_batch(x_train, y_train, 
                                                  batch_size = batch_size),
                      steps_per_epoch = train_samples // batch_size,
                      epochs = epochs,
                      validation_data = generate_batch(x_val, y_val, 
                                                      batch_size = batch_size),
                      validation_steps = val_samples // batch_size,
                      callbacks = rlronp
                      )

Epoch 1/25


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


93/93 [==============================] - 87s 855ms/step - loss: -2.7020e-05 - acc: 0.0000e+00 - precision_8: 0.0000e+00 - recall_8: 0.0000e+00 - auc_5: 0.5000 - val_loss: -3.0172e-05 - val_acc: 0.0000e+00 - val_precision_8: 0.0000e+00 - val_recall_8: 0.0000e+00 - val_auc_5: 0.5000 - lr: 1.0000e-05
Epoch 2/25
93/93 [==============================] - ETA: 0s - loss: -5.8007e-05 - acc: 0.0015 - precision_8: 0.0000e+00 - recall_8: 0.0000e+00 - auc_5: 0.5000

KeyboardInterrupt: ignored

In [ ]:
model.save_weights('lstm_translation_weights.h5')

In [ ]:
# Encode the input sequence to get the 'thought vectors'
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape = (latent_dim, ))
decoder_state_input_c = Input(shape = (latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from 
# the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, 
                                                    initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate 
# probability distribution over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [ ]:
def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1))

  # Populate the first character of target sequence with the start character.
  target_seq[0, 0] = target_token_index['START_']

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1)
  stop_condition = False
  decoded_sentence = ''

  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    
    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += ' '+ sampled_char

    # Exit condition: either hit max length
    # or find stop character
    if (sampled_char == '_END' or len(decoded_sentence) > 50):
      stop_condition = True

    # Update the target sequence (of lenfth 1).
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = sampled_token_index

    # update states
    states_value = [h, c]

  return decoded_sentence

In [ ]:
test_gen = generate_batch(x_test, y_test, batch_size = 1)

In [ ]:
for k in range(5):

  (input_seq, actual_output), _ = next(test_gen)
  decoded_sentence = decode_sequence(input_seq)
  print('Input English sentence:', x_test[k : k + 1].values[0])
  print("Actual Hindi Translation:", y_test[k : k + 1].values[0][6:-4])
  print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
classification_report(decoded_sentence, y_test, target_names=['English', 'Hindi'])